In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
from keras.models import load_model
import urllib.request
import re

Using TensorFlow backend.


## Find odds for 7 days at a time.

### Source 1: soccerbase

In [2]:
import gzip
import urllib.request
games = []

In [3]:

# url_string2 = None
# with urllib.request.urlopen('http://www.soccerbase.com/matches/home.sd?type=2') as response:
#     gzipFile = gzip.GzipFile(fileobj=response)
#     url_string2 = gzipFile.read().decode("UTF-8")
#     url_string2 = re.search('(?<=soccer.gamelist.Timekeeper.addGames\({).*?(?=soccer.gamelist)', url_string2)[0]


In [4]:
# matches = re.finditer('(?<=\=)(.*?)\_V\_(.*?)\=(.*?)\"', url_string2)
# for match in matches:
#     games.append([match[3],re.sub("_", " ", match[1]).lower(),re.sub("_", " ", match[2]).lower()])

### Source 2: supersport

In [5]:
# response = urllib.request.urlopen('https://www.supersport.com/football/fixtures')
# url_string = response.read()
# url_string = url_string.decode("UTF-8")
# url_string = re.search('(?<=FIXTURES)(.|\s)*', url_string)[0]

In [6]:
# dates = re.finditer("(?<=Header\">)(.*?)(?=class=\"table)", url_string)

# for day in dates:
    
#     date = re.search('.*?(?=</td>)',day[0])[0]
#     matches = re.finditer("(?<=>)(..:..)(.*?)(?=</td><td class)",day[0])
    
#     for match in matches:
#         time_of_match = match[0][:5]
#         teams_re = re.finditer("(?<=st\'>)(.*?)(?=</td>)", match[0])
#         game = [date]
#         for team in teams_re:
            
#             game.append(team[0].lower())
#         games.append(game)

### Source 3: thefishy

In [7]:
url_string3 = None
with urllib.request.urlopen('https://thefishy.co.uk/nextweeksfixtures.php') as response:
    url_string3 = response.read().decode("UTF-8") + " 12/12</th>"

In [8]:
dates = re.finditer("(\ ../..<(.|\s)*?)(?=\ \w\w/\w\w</th>)", url_string3)

for date in dates:
    day = date[0][1:6] + "/18"
    matches = re.finditer("(?<=\'>)(.*?)\ v\ (.*?)(?=<)", date[0])
    for match in matches:
        away_team = match[2]
        home_team = re.search('(?<=team2)(.*?>)(.*?)(?=$)',match[1])[2]
        
#         print(home_team, away_team)
        
        games.append([day, home_team.lower(), away_team.lower()])


In [9]:
%run update_stash.py

unzipping  ../../Football_Data/Zipstash\data_0.zip
unzipping  ../../Football_Data/Zipstash\data_1.zip
unzipping  ../../Football_Data/Zipstash\data_10.zip
unzipping  ../../Football_Data/Zipstash\data_11.zip
unzipping  ../../Football_Data/Zipstash\data_12.zip
unzipping  ../../Football_Data/Zipstash\data_13.zip
unzipping  ../../Football_Data/Zipstash\data_14.zip
unzipping  ../../Football_Data/Zipstash\data_15.zip
unzipping  ../../Football_Data/Zipstash\data_16.zip
unzipping  ../../Football_Data/Zipstash\data_17.zip
unzipping  ../../Football_Data/Zipstash\data_18.zip
unzipping  ../../Football_Data/Zipstash\data_19.zip
unzipping  ../../Football_Data/Zipstash\data_2.zip
unzipping  ../../Football_Data/Zipstash\data_20.zip
unzipping  ../../Football_Data/Zipstash\data_21.zip
unzipping  ../../Football_Data/Zipstash\data_22.zip
unzipping  ../../Football_Data/Zipstash\data_23.zip
unzipping  ../../Football_Data/Zipstash\data_24.zip
unzipping  ../../Football_Data/Zipstash\data_3.zip
unzipping  ../..

In [10]:
all_seasons = []
for filename in os.listdir('C:/Users/bfesc/Documents/Football_Data/Stash/'):
    #if filename[0] == 'B':
    with open("C:/Users/bfesc/Documents/Football_Data/Stash/" + filename) as file:
        season = file.read().rstrip().split('\n')
        for i in range(len(season)):
            season[i] = season[i].rstrip(',').split(',')
            for j in range(len(season[i])):
                season[i][j] = season[i][j].lower()
        all_seasons.append(season)

In [11]:
smallest_season = np.argmin([len(season[0]) for season in all_seasons])
columns = ['div', 'date', 'hometeam', 'awayteam', 'fthg', 'ftag', 'ftr']

all_data = pd.DataFrame(columns=columns)
for season in all_seasons:
    if not np.in1d(columns, season[0]).all():
        continue
    if len(season[0]) != max([len(s) for s in season]):
        for i in range(len(season)):
            season[i] = season[i][:len(season[0])]
    df = pd.DataFrame(season[1:])
    df.columns = season[0]
    all_data = all_data.append(df[columns])

all_data['date'] = pd.to_datetime(all_data['date'], dayfirst=True)
print(all_data.shape)
all_data.index = all_data['date']
all_data = all_data.drop(columns=['date','div'])
for column in all_data.columns:
    all_data = all_data[pd.notnull(all_data[column])]
all_data = all_data[pd.notnull(all_data.index)]
all_data = all_data.iloc[np.where(all_data['fthg'] != "")]
all_data = all_data.sort_index(ascending=False)
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'a')] = -1
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'd')] = 0
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'h')] = 1
all_data['fthg'] = all_data['fthg'].astype(int)
all_data['ftag'] = all_data['ftag'].astype(int)
print(all_data.shape)

(198678, 7)
(178793, 5)


In [12]:
from difflib import SequenceMatcher

def unknown_team(home_team, away_team, team_dict, unknown = "away"):

    if unknown == "away":
        team_scores = []
        teams = team_dict[home_team]['awayteam'] #Away or HomeTeam doesnt really matter 
        
        for team in teams:
            team_scores.append(SequenceMatcher(None,team,away_team).ratio())
        
        team_scores = np.argsort(team_scores)
        
        away_team = teams[team_scores[-1]]
    
    else:
        teams = team_dict[away_team]['hometeam']
        team_scores = []
            
        for team in teams:
            team_scores.append(SequenceMatcher(None,team,home_team).ratio())
        
        team_scores = np.argsort(team_scores)
        
        home_team = teams[team_scores[-1]]
        
        #if team_scores[-1] < 0.4
        
    return home_team, away_team


In [13]:
def make_team_dicts(data):
    all_teams = np.unique(np.append(data['hometeam'].str.lower(), data['awayteam'].str.lower()))
    team_dict = {}
    
    for team in all_teams:
        team_dict[team] = data.iloc[np.where(np.logical_or(data['hometeam'] == team, 
                                                      data['awayteam'] == team))].sort_index(ascending=False)
        
    return team_dict

In [14]:
team_dict = make_team_dicts(all_data)

In [15]:
# Sanity check
unknown_team("dortmund", "bayern münchen", team_dict)

('dortmund', 'bayern munich')

In [16]:
def make_live_data(live_data, team_dict):


    training_data = pd.DataFrame(columns=['Date','HomeTeam', 'AwayTeam', 'DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
                                            'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
                                          'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame', 'Result'])


    history = pd.Timedelta('90d')
    year = pd.Timedelta('365d')
    for i in tqdm(range(len(live_data))):#
        game = live_data[i]
        home_team = game[1]
        away_team = game[2]
        game_date = pd.to_datetime(game[0]) 

        if home_team not in team_dict.keys() and away_team not in team_dict.keys():
            continue
        elif home_team not in team_dict.keys():
            home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="home")
        elif away_team not in team_dict.keys():
            home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="away")
        
        ht_games = team_dict[home_team].loc[game_date - pd.Timedelta('1h'):game_date - year]
        at_games = team_dict[away_team].loc[game_date - pd.Timedelta('1h'):game_date - year]

        #result = game['FTR']
        result = 0

        ht_hgames = ht_games.iloc[np.where(ht_games['hometeam'] == home_team)]
        ht_agames = ht_games.iloc[np.where(ht_games['awayteam'] == home_team)]

        at_hgames = at_games.iloc[np.where(at_games['hometeam'] == away_team)]
        at_agames = at_games.iloc[np.where(at_games['awayteam'] == away_team)]


        if len(ht_hgames) > 1 and len(at_agames) > 1:

            ht_time_diff = np.log((game_date - ht_games.index).days.values + 1)
            at_time_diff = np.log((game_date - at_games.index).days.values + 1)

            ht_h_td = ht_time_diff[np.where(ht_games['hometeam'] == home_team)]
            ht_a_td = ht_time_diff[np.where(ht_games['awayteam'] == home_team)]
            at_h_td = at_time_diff[np.where(at_games['hometeam'] == away_team)]
            at_a_td = at_time_diff[np.where(at_games['awayteam'] == away_team)]


            it_games = np.append(np.where(ht_hgames['awayteam'] == away_team)[0],
                                 np.where(ht_agames['hometeam'] == away_team)[0])

            litgr = 0
            if len(it_games) > 0:
                litgr += np.mean((ht_games['fthg'].iloc[it_games].values - ht_games['ftag'].iloc[it_games].values) / ht_time_diff[it_games])

            dslhtm = ht_time_diff[0]
            dslatm = at_time_diff[0]

            ht_form = np.mean(np.append(((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td),
                              (ht_agames['ftag'].values - ht_agames['fthg'].values) / ht_a_td))
            at_form = np.mean(np.append((at_hgames['fthg'].values - at_hgames['ftag'].values) / at_h_td,
                              (at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td))

            ht_hform = np.mean((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td)
            at_aform = np.mean((at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td)

            hto = np.mean(np.append(ht_hgames['fthg'].values / ht_h_td, ht_agames['ftag'].values / ht_a_td))
            htd = -np.mean(np.append(ht_hgames['ftag'].values / ht_h_td, ht_agames['fthg'].values /ht_a_td))

            ato = np.mean(np.append(at_hgames['fthg'] / at_h_td, at_agames['ftag'].values / at_a_td))
            atd = -np.mean(np.append(at_hgames['ftag'].values / at_h_td, at_agames['fthg'].values / at_a_td))


            training_data = training_data.append(pd.DataFrame([[game_date,home_team, away_team, dslhtm, dslatm, ht_form, at_form, ht_hform, at_aform,
                                                                hto, htd, ato, atd, litgr, result]], columns=training_data.columns))

    return training_data
    
    

In [17]:
training_data = make_live_data(games,team_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 295/295 [00:03<00:00, 92.64it/s]


In [18]:
training_data = training_data.reset_index(drop=True)
training_data.mean()

DaysSinceLastHTMatch    3.996519
DaysSinceLastATMatch    3.994774
HomeTeamForm            0.016379
AwayTeamForm            0.015977
HomeTeamHomeForm        0.076933
AwayTeamAwayForm       -0.048920
HomeTeamOffense         0.266265
HomeTeamDefense        -0.249886
AwayTeamOffense         0.270365
AwayTeamDefense        -0.254388
dtype: float64

In [19]:
training_data.shape

(127, 15)

In [20]:
import tensorflow as tf

X = training_data[['DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
                                            'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
                                          'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame']]
with tf.device('/cpu:0'):
    model = load_model('football_model.h5')

In [21]:
pred = None
with tf.device('/cpu:0'):
    pred = model.predict(X)

In [22]:
training_data.head()

,Date,HomeTeam,AwayTeam,DaysSinceLastHTMatch,DaysSinceLastATMatch,HomeTeamForm,AwayTeamForm,HomeTeamHomeForm,AwayTeamAwayForm,HomeTeamOffense,HomeTeamDefense,AwayTeamOffense,AwayTeamDefense,LastInterTeamGame,Result
0,2018-10-05,west ham,man united,5.036953,5.043425,-0.099420,0.124575,-0.076116,0.072627,0.237859,-0.337279,0.286883,-0.162308,0,0
1,2018-10-05,charlton,shrewsbury,5.036953,5.036953,0.017124,0.055666,0.050735,0.020561,0.203544,-0.186420,0.223827,-0.168161,-0.00119304,0
2,2018-10-05,anderlecht,standard,5.342334,5.342334,0.050130,0.084477,0.129732,0.054210,0.321939,-0.271809,0.315053,-0.230576,0.183596,0
3,2018-10-05,charleroi,club brugge,5.342334,5.342334,0.065869,0.167265,0.087704,0.018246,0.251693,-0.185824,0.387769,-0.220505,0,0
4,2018-10-05,genk,gent,5.342334,5.342334,0.069101,0.176517,0.050364,0.071815,0.239016,-0.169915,0.305028,-0.128511,-0.187184,0


In [23]:
from calendar import month_abbr

threshold = 10
home_bet = 10.5
away_bet = -3
recs = 0

for i in range(len(pred)):
    date = training_data['Date'].iloc[i]
    if pred[i] > home_bet:
        print("{}\n{} vs {}:".format(str(date.month) + ". " + month_abbr[date.day],training_data['HomeTeam'].iloc[i],training_data['AwayTeam'].iloc[i]))
        print("HOME if odds above: {}\n".format(2.0))
        recs+=1
    elif pred[i] < away_bet:
        print("{}\n{} vs {}:".format(str(date.month) + ". " + month_abbr[date.day],training_data['HomeTeam'].iloc[i],training_data['AwayTeam'].iloc[i]))
        print("AWAY if odds above: {}\n".format(1.5))
        recs+=1
        
print("Matches recommended: ", recs)

10. May
charlton vs shrewsbury:
HOME if odds above: 2.0

10. May
anderlecht vs standard:
HOME if odds above: 2.0

10. May
charleroi vs club brugge:
HOME if odds above: 2.0

11. May
derby vs fulham:
HOME if odds above: 2.0

11. May
livingston vs dundee united:
HOME if odds above: 2.0

11. May
chateauroux vs brest:
HOME if odds above: 2.0

11. May
clermont vs paris fc:
HOME if odds above: 2.0

11. May
ajaccio vs valenciennes:
HOME if odds above: 2.0

11. May
le havre vs tours:
HOME if odds above: 2.0

11. May
lens vs auxerre:
HOME if odds above: 2.0

11. May
nancy vs orleans:
HOME if odds above: 2.0

11. May
reims vs nimes:
HOME if odds above: 2.0

11. May
aves vs aves:
HOME if odds above: 2.0

12. May
middlesbrough vs aston villa:
HOME if odds above: 2.0

12. May
lincoln vs exeter:
HOME if odds above: 2.0

12. May
coventry vs notts county:
HOME if odds above: 2.0

12. May
dundee vs partick:
HOME if odds above: 2.0

12. May
motherwell vs hamilton:
HOME if odds above: 2.0

12. May
st john

IndexError: list index out of range

# Remember:
When using a confidence threshold of around 10, the model yields a return of around 3% in testing. Much higher or lower, and the return is practically zero. This could be due to an unwillingness from the bookmakers to realistic odds for dominant sides. Possibly because people overestimate the probability of an underdog win, and so find lower odds unrealistic and unappealing.<br><br>
An optimal strategy with regards to risk remains to be discovered, but so far, low-risk strategies (batches of 30 or more matches) seem to be the best course. Constant bets also seem to be doing well, although this may not be statistically significant.

### UPDATE:
Taking the product of odds and score (negative in the away case), and thresholding at 21 for home games and 7 for away games yields more profit.<br>


### UPDATE II:
Examining contour plots, it appears that an even better strategy consists of using a home win certainty threshold of 10.5 and an away win certainty threshold of -3, as well as using home and away odd thresholds of 2 and 1.5 respectively (this relationship should be examined closer). This yields 10% in testing, but I may be pushing how far I can tailor the parameters to the test data. Additionally, batch sizes between 10 and 30 are preferable.